In [10]:
# try to use GPU to accelerate the training process
import torch
import torch.nn as nn
from d2l import torch as d2l

In [14]:
import torch.utils.data as Data

batch_size = 256

train_data, test_data = d2l.load_data_fashion_mnist(batch_size=256)

train_dataloader = Data.DataLoader(train_data, batch_size=64)
test_dataloader = Data.DataLoader(test_data, batch_size=64)

In [20]:
myGPU = torch.device('cuda:0')

net = nn.Sequential(nn.Flatten(), nn.Linear(784, 10))
net = net.to(device = myGPU)

In [24]:
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size=256)

In [ ]:
batch_size = 256

train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size=256)

net = nn.Sequential(nn.Flatten(), nn.Linear(784, 10))
net = net.to(device = myGPU)

def init_weights(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, std=0.01)

net.apply(init_weights)

loss = nn.CrossEntropyLoss()
optimiser = torch.optim.SGD(net.parameters(), lr=0.2)

num_epochs = 500

In [ ]:
# write a trainer function that can do the training process on GPU
def train(net, train_iter, test_iter, loss, num_epochs, optimiser, device):
    net = net.to(device)
    print("training on ", device)
    batch_count = 0
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n = 0.0, 0.0, 0
        for X, y in train_iter:
            X = X.to(device)
            y = y.to(device)
            y_hat = net(X)
            l = loss(y_hat, y)
            optimiser.zero_grad()
            l.backward()
            optimiser.step()
            train_l_sum += l.cpu().item()
            train_acc_sum += (y_hat.argmax(dim=1) == y).sum().cpu().item()
            n += y.shape[0]
            batch_count += 1
            
        
        # print every 100 epoch
        if (epoch + 1) % 100 == 0:
            test_acc = d2l.evaluate_accuracy_gpu(net, test_iter)
            print("epoch %d, loss %.4f, train acc %.3f, test acc %.3f" % (epoch + 1, train_l_sum / batch_count, train_acc_sum / n, test_acc))

In [ ]:
train(net, train_iter, test_iter, loss, num_epochs, optimiser, myGPU)